# Initialisation

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2regional as br
import numpy as np
from bw2regional.lca.base_class import RegionalizationBase
import itertools
import numpy as np
from bw2calc.lca import LCA
from bw2calc.matrices import MatrixBuilder as builder
from bw2data import Database, Method, databases, get_activity, methods
from scipy.sparse import coo_matrix, csr_matrix

In [5]:
bd.projects.set_current("Brightway regional tutorial")

# Printing the calculated intersections

In [6]:
list(br.intersections)

[('RoW', 'Freshwater-acidification-country'),
 ('Freshwater-acidification-country', 'RoW'),
 ('world', 'Freshwater-acidification-country'),
 ('Freshwater-acidification-country', 'world'),
 ('RoE', 'Freshwater-acidification-country'),
 ('Freshwater-acidification-country', 'RoE'),
 ('ecoinvent', 'Freshwater-acidification-country'),
 ('Freshwater-acidification-country', 'ecoinvent')]

# Under the hood

Define the functional unit : 1 MJ of biodiesel from rapeseed

In [7]:
biodiesel_rapeseed = [activity for activity in bd.Database('BW2regional_tutorial') if "Esterification - Rapeseed - MJ" in activity['name']][0]

In [8]:
UF = {biodiesel_rapeseed:1}

Finding the regionalized method

In [9]:
method = [methods for methods in bd.methods if 'IMPACT World+ Midpoint 2.1 for ecoinvent v3.10, regionalized at the country scale' in methods[0]][0]

In [11]:
lca_biodiesel_rapeseed = br.TwoSpatialScalesLCA(demand = UF, method = method)

In [12]:
lca_biodiesel_rapeseed.lci()

In [13]:
lca_biodiesel_rapeseed.lcia()

In [14]:
lca_biodiesel_rapeseed.score

2.0197827506680342e-05

Four matrices are composing this class

In [15]:
lca_biodiesel_rapeseed.inv_mapping_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 23527 stored elements and shape (23527, 663)>

In [16]:
lca_biodiesel_rapeseed.normalization_matrix

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 652 stored elements and shape (663, 663)>

In [17]:
lca_biodiesel_rapeseed.geo_transform_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 68240 stored elements and shape (663, 228)>

In [18]:
lca_biodiesel_rapeseed.reg_cf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1824 stored elements and shape (228, 2648)>

## Inventory mapping matrix

In [19]:
inv_mapping_params, _, inv_spatial_dict, inv_mapping_matrix = builder.build(
            paths=[Database(x).filepath_geomapping() for x in lca_biodiesel_rapeseed.databases],
            data_label="amount",
            row_id_label="activity",
            row_index_label="row",
            col_id_label="geo",
            col_index_label="col",
            row_dict=lca_biodiesel_rapeseed._activity_dict,
        )

### Computing the arguments

Path

In [20]:
[Database(x).filepath_geomapping() for x in lca_biodiesel_rapeseed.databases]

['C:\\Users\\rogyn\\AppData\\Local\\pylca\\Brightway3\\Brightway-regional-tutorial.5e564e7456aa35ab7701e0b9a480b6e3\\processed\\biosphere3.5d405d7124beac53e64a4cd0f9e012f4.geomapping.npy',
 'C:\\Users\\rogyn\\AppData\\Local\\pylca\\Brightway3\\Brightway-regional-tutorial.5e564e7456aa35ab7701e0b9a480b6e3\\processed\\BW2regional_tutorial.6ff0e1217f0ea031e883e33fff226e6d.geomapping.npy',
 'C:\\Users\\rogyn\\AppData\\Local\\pylca\\Brightway3\\Brightway-regional-tutorial.5e564e7456aa35ab7701e0b9a480b6e3\\processed\\ecoinvent-3101-cutoff.333d1f9be3c0d7d112bc27950c1520e2.geomapping.npy']

row_dict

In [22]:
lca_biodiesel_rapeseed._activity_dict

{4363: 0,
 4364: 1,
 4365: 2,
 4366: 3,
 4367: 4,
 4368: 5,
 4369: 6,
 4370: 7,
 4371: 8,
 4372: 9,
 4373: 10,
 4374: 11,
 4375: 12,
 4376: 13,
 4377: 14,
 4378: 15,
 4379: 16,
 4380: 17,
 4381: 18,
 4382: 19,
 4383: 20,
 4384: 21,
 4385: 22,
 4386: 23,
 4387: 24,
 4388: 25,
 4389: 26,
 4390: 27,
 4391: 28,
 4392: 29,
 4393: 30,
 4394: 31,
 4395: 32,
 4396: 33,
 4397: 34,
 4398: 35,
 4399: 36,
 4400: 37,
 4401: 38,
 4402: 39,
 4403: 40,
 4404: 41,
 4405: 42,
 4406: 43,
 4407: 44,
 4408: 45,
 4409: 46,
 4410: 47,
 4411: 48,
 4412: 49,
 4413: 50,
 4414: 51,
 4415: 52,
 4416: 53,
 4417: 54,
 4418: 55,
 4419: 56,
 4420: 57,
 4421: 58,
 4422: 59,
 4423: 60,
 4424: 61,
 4425: 62,
 4426: 63,
 4427: 64,
 4428: 65,
 4429: 66,
 4430: 67,
 4431: 68,
 4432: 69,
 4433: 70,
 4434: 71,
 4435: 72,
 4436: 73,
 4437: 74,
 4438: 75,
 4439: 76,
 4440: 77,
 4441: 78,
 4442: 79,
 4443: 80,
 4444: 81,
 4445: 82,
 4446: 83,
 4447: 84,
 4448: 85,
 4449: 86,
 4450: 87,
 4451: 88,
 4452: 89,
 4453: 90,
 4454: 91

### Computing the outputs

In [23]:
inv_mapping_params

array([(27886, 144, 23523,  66, 0, 1., nan, nan, nan, nan, nan, False),
       (27887, 144, 23524,  66, 0, 1., nan, nan, nan, nan, nan, False),
       (27888, 144, 23525,  66, 0, 1., nan, nan, nan, nan, nan, False),
       ...,
       (27883, 986, 23520, 291, 0, 1., nan, nan, nan, nan, nan, False),
       (27884, 986, 23521, 291, 0, 1., nan, nan, nan, nan, nan, False),
       (27885,   1, 23522,   0, 0, 1., nan, nan, nan, nan, nan, False)],
      dtype=[('activity', '<u4'), ('geo', '<u4'), ('row', '<u4'), ('col', '<u4'), ('uncertainty_type', 'u1'), ('amount', '<f4'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')])

In [24]:
inv_spatial_dict

{1: 0,
 2: 1,
 4: 2,
 11: 3,
 16: 4,
 17: 5,
 18: 6,
 20: 7,
 22: 8,
 23: 9,
 26: 10,
 28: 11,
 29: 12,
 30: 13,
 31: 14,
 32: 15,
 33: 16,
 34: 17,
 36: 18,
 37: 19,
 39: 20,
 40: 21,
 43: 22,
 44: 23,
 47: 24,
 54: 25,
 56: 26,
 57: 27,
 64: 28,
 65: 29,
 66: 30,
 67: 31,
 69: 32,
 70: 33,
 72: 34,
 73: 35,
 75: 36,
 76: 37,
 78: 38,
 79: 39,
 80: 40,
 84: 41,
 85: 42,
 88: 43,
 90: 44,
 96: 45,
 98: 46,
 99: 47,
 102: 48,
 104: 49,
 105: 50,
 109: 51,
 113: 52,
 114: 53,
 115: 54,
 123: 55,
 128: 56,
 130: 57,
 131: 58,
 132: 59,
 134: 60,
 136: 61,
 138: 62,
 139: 63,
 140: 64,
 143: 65,
 144: 66,
 145: 67,
 147: 68,
 148: 69,
 151: 70,
 153: 71,
 157: 72,
 158: 73,
 161: 74,
 162: 75,
 163: 76,
 164: 77,
 165: 78,
 166: 79,
 167: 80,
 168: 81,
 169: 82,
 171: 83,
 172: 84,
 174: 85,
 175: 86,
 177: 87,
 178: 88,
 187: 89,
 189: 90,
 192: 91,
 197: 92,
 200: 93,
 202: 94,
 207: 95,
 208: 96,
 215: 97,
 218: 98,
 220: 99,
 221: 100,
 225: 101,
 230: 102,
 231: 103,
 232: 104,
 236: 

In [25]:
print(inv_mapping_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 23527 stored elements and shape (23527, 663)>
  Coords	Values
  (0, 187)	1.0
  (1, 114)	1.0
  (2, 196)	1.0
  (3, 23)	1.0
  (4, 241)	1.0
  (5, 83)	1.0
  (6, 424)	1.0
  (7, 47)	1.0
  (8, 61)	1.0
  (9, 79)	1.0
  (10, 66)	1.0
  (11, 364)	1.0
  (12, 130)	1.0
  (13, 61)	1.0
  (14, 130)	1.0
  (15, 61)	1.0
  (16, 291)	1.0
  (17, 291)	1.0
  (18, 291)	1.0
  (19, 81)	1.0
  (20, 433)	1.0
  (21, 433)	1.0
  (22, 183)	1.0
  (23, 196)	1.0
  (24, 0)	1.0
  :	:
  (23502, 424)	1.0
  (23503, 424)	1.0
  (23504, 241)	1.0
  (23505, 319)	1.0
  (23506, 319)	1.0
  (23507, 177)	1.0
  (23508, 0)	1.0
  (23509, 0)	1.0
  (23510, 0)	1.0
  (23511, 139)	1.0
  (23512, 438)	1.0
  (23513, 96)	1.0
  (23514, 207)	1.0
  (23515, 364)	1.0
  (23516, 215)	1.0
  (23517, 61)	1.0
  (23518, 424)	1.0
  (23519, 291)	1.0
  (23520, 291)	1.0
  (23521, 291)	1.0
  (23522, 0)	1.0
  (23523, 66)	1.0
  (23524, 66)	1.0
  (23525, 66)	1.0
  (23526, 66)	1.0


## Regionalized CF matrix

For the regionalized characterization matrix, the mapping of the column correspond to the locations of the inventory, as it is supposed that the LCI and LCIA share the same spatial scale.

In [27]:
reg_cf_params, ia_spatial_dict, _, reg_cf_matrix = builder.build(
            paths=[Method(lca_biodiesel_rapeseed.method).filepath_processed()],
            data_label="amount",
            row_id_label="geo",
            row_index_label="row",
            col_id_label="flow",
            col_index_label="col",
            col_dict=lca_biodiesel_rapeseed._biosphere_dict,
        )

### Computing the arguments

Paths

In [28]:
[Method(lca_biodiesel_rapeseed.method).filepath_processed()]

['C:\\Users\\rogyn\\AppData\\Local\\pylca\\Brightway3\\Brightway-regional-tutorial.5e564e7456aa35ab7701e0b9a480b6e3\\processed\\impact-world-midpoint-21-for-ecoinvent-v310-regionalized-at-the-country-scalemf.b310c738aba0c903aa5c230d4b99a0a6.npy']

col_dict

In [29]:
lca_biodiesel_rapeseed._biosphere_dict

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 13: 11,
 14: 12,
 16: 13,
 17: 14,
 18: 15,
 19: 16,
 21: 17,
 22: 18,
 26: 19,
 27: 20,
 28: 21,
 32: 22,
 33: 23,
 34: 24,
 35: 25,
 36: 26,
 38: 27,
 39: 28,
 40: 29,
 41: 30,
 42: 31,
 43: 32,
 44: 33,
 45: 34,
 46: 35,
 47: 36,
 52: 37,
 53: 38,
 54: 39,
 55: 40,
 56: 41,
 58: 42,
 60: 43,
 62: 44,
 63: 45,
 64: 46,
 65: 47,
 66: 48,
 67: 49,
 69: 50,
 70: 51,
 71: 52,
 72: 53,
 74: 54,
 75: 55,
 76: 56,
 77: 57,
 78: 58,
 80: 59,
 81: 60,
 82: 61,
 84: 62,
 87: 63,
 88: 64,
 89: 65,
 90: 66,
 91: 67,
 92: 68,
 93: 69,
 96: 70,
 97: 71,
 99: 72,
 100: 73,
 101: 74,
 102: 75,
 103: 76,
 104: 77,
 105: 78,
 106: 79,
 107: 80,
 108: 81,
 109: 82,
 110: 83,
 111: 84,
 112: 85,
 113: 86,
 120: 87,
 122: 88,
 123: 89,
 124: 90,
 126: 91,
 127: 92,
 129: 93,
 131: 94,
 132: 95,
 133: 96,
 134: 97,
 135: 98,
 136: 99,
 137: 100,
 138: 101,
 139: 102,
 141: 103,
 142: 104,
 143: 105,
 144: 106,
 145: 107,
 146

### Outputs

In [30]:
reg_cf_params

array([(1461,   1,   0,  926, 0, 0.2915794 , 0.2915794 , nan, nan, nan, nan, False),
       (4201,   1,   0, 2523, 0, 0.39303803, 0.39303803, nan, nan, nan, nan, False),
       (3384,   1,   0, 1951, 0, 0.6023909 , 0.6023909 , nan, nan, nan, nan, False),
       ...,
       (4292, 559, 227, 2580, 0, 0.10204792, 0.10204792, nan, nan, nan, nan, False),
       ( 939, 559, 227,  663, 0, 0.1075194 , 0.1075194 , nan, nan, nan, nan, False),
       (1650, 559, 227, 1027, 0, 0.12752804, 0.12752804, nan, nan, nan, nan, False)],
      dtype=[('flow', '<u4'), ('geo', '<u4'), ('row', '<u4'), ('col', '<u4'), ('uncertainty_type', 'u1'), ('amount', '<f4'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')])

In [31]:
ia_spatial_dict

{1: 0,
 332: 1,
 333: 2,
 334: 3,
 335: 4,
 336: 5,
 337: 6,
 338: 7,
 339: 8,
 340: 9,
 341: 10,
 342: 11,
 343: 12,
 344: 13,
 345: 14,
 346: 15,
 347: 16,
 348: 17,
 349: 18,
 350: 19,
 351: 20,
 352: 21,
 353: 22,
 354: 23,
 355: 24,
 356: 25,
 357: 26,
 358: 27,
 359: 28,
 360: 29,
 361: 30,
 362: 31,
 363: 32,
 364: 33,
 365: 34,
 366: 35,
 367: 36,
 368: 37,
 369: 38,
 370: 39,
 371: 40,
 372: 41,
 373: 42,
 374: 43,
 375: 44,
 376: 45,
 377: 46,
 378: 47,
 379: 48,
 380: 49,
 381: 50,
 382: 51,
 383: 52,
 384: 53,
 385: 54,
 386: 55,
 387: 56,
 388: 57,
 389: 58,
 390: 59,
 391: 60,
 392: 61,
 393: 62,
 394: 63,
 395: 64,
 396: 65,
 397: 66,
 398: 67,
 399: 68,
 400: 69,
 401: 70,
 402: 71,
 403: 72,
 404: 73,
 405: 74,
 406: 75,
 407: 76,
 408: 77,
 409: 78,
 410: 79,
 411: 80,
 412: 81,
 413: 82,
 414: 83,
 415: 84,
 416: 85,
 417: 86,
 418: 87,
 419: 88,
 420: 89,
 421: 90,
 422: 91,
 423: 92,
 424: 93,
 425: 94,
 426: 95,
 427: 96,
 428: 97,
 429: 98,
 430: 99,
 431: 100,
 

In [32]:
print(reg_cf_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1824 stored elements and shape (228, 2648)>
  Coords	Values
  (0, 663)	0.6948829889297485
  (0, 926)	0.29157939553260803
  (0, 1024)	0.6669330596923828
  (0, 1027)	1.0
  (0, 1031)	0.6531468629837036
  (0, 1951)	0.6023908853530884
  (0, 2523)	0.3930380344390869
  (0, 2580)	0.8001998066902161
  (1, 663)	0.006059170234948397
  (1, 926)	0.0034653323236852884
  (1, 1024)	0.002214280189946294
  (1, 1027)	0.003320093499496579
  (1, 1031)	0.0021685087122023106
  (1, 1951)	0.007159232161939144
  (1, 2523)	0.004671137314289808
  (1, 2580)	0.0026567380409687757
  (2, 663)	0.5112569332122803
  (2, 926)	0.18449528515338898
  (2, 1024)	0.5374437570571899
  (2, 1027)	0.8058435916900635
  (2, 1031)	0.5263342261314392
  (2, 1951)	0.38115957379341125
  (2, 2523)	0.24869269132614136
  (2, 2580)	0.644835889339447
  (3, 663)	0.38673555850982666
  :	:
  (224, 2580)	0.07066918164491653
  (225, 663)	0.13398371636867523
  (225, 926)	0.03147583454847

In LCIA methods using site-specific CFs, the CFs are available for different locations. That is a good new 🥳

## Geographic transform matrix

The columns of the geographic transform matrix consists in the location of the LCIA and the rows consists in the location of the LCI.

In [35]:
geo_transform_params, _, _, geo_transform_matrix = builder.build(
            paths=[
                br.Intersection(name).filepath_processed()
                for name in lca_biodiesel_rapeseed.needed_intersections()
            ],
            data_label="amount",
            row_id_label="geo_inv",
            row_index_label="row",
            col_id_label="geo_ia",
            col_index_label="col",
            row_dict=lca_biodiesel_rapeseed.inv_spatial_dict,
            col_dict=lca_biodiesel_rapeseed.ia_spatial_dict,
        )

### Computing the arguments

paths

In [36]:
[
                br.Intersection(name).filepath_processed()
                for name in lca_biodiesel_rapeseed.needed_intersections()
            ]

['C:\\Users\\rogyn\\AppData\\Local\\pylca\\Brightway3\\Brightway-regional-tutorial.5e564e7456aa35ab7701e0b9a480b6e3\\processed\\rowf.3db731c4a26574f66b87e7c94118f9a5.npy',
 'C:\\Users\\rogyn\\AppData\\Local\\pylca\\Brightway3\\Brightway-regional-tutorial.5e564e7456aa35ab7701e0b9a480b6e3\\processed\\roef.6132de82245209b048334417deae6b14.npy',
 'C:\\Users\\rogyn\\AppData\\Local\\pylca\\Brightway3\\Brightway-regional-tutorial.5e564e7456aa35ab7701e0b9a480b6e3\\processed\\worldf.435e681695096a98008b7a5108e776f6.npy',
 'C:\\Users\\rogyn\\AppData\\Local\\pylca\\Brightway3\\Brightway-regional-tutorial.5e564e7456aa35ab7701e0b9a480b6e3\\processed\\ecoinventf.2d8d16a5e75473e72a7a930cc3ba01fa.npy']

row_dict

In [37]:
lca_biodiesel_rapeseed.inv_spatial_dict

{1: 0,
 2: 1,
 4: 2,
 11: 3,
 16: 4,
 17: 5,
 18: 6,
 20: 7,
 22: 8,
 23: 9,
 26: 10,
 28: 11,
 29: 12,
 30: 13,
 31: 14,
 32: 15,
 33: 16,
 34: 17,
 36: 18,
 37: 19,
 39: 20,
 40: 21,
 43: 22,
 44: 23,
 47: 24,
 54: 25,
 56: 26,
 57: 27,
 64: 28,
 65: 29,
 66: 30,
 67: 31,
 69: 32,
 70: 33,
 72: 34,
 73: 35,
 75: 36,
 76: 37,
 78: 38,
 79: 39,
 80: 40,
 84: 41,
 85: 42,
 88: 43,
 90: 44,
 96: 45,
 98: 46,
 99: 47,
 102: 48,
 104: 49,
 105: 50,
 109: 51,
 113: 52,
 114: 53,
 115: 54,
 123: 55,
 128: 56,
 130: 57,
 131: 58,
 132: 59,
 134: 60,
 136: 61,
 138: 62,
 139: 63,
 140: 64,
 143: 65,
 144: 66,
 145: 67,
 147: 68,
 148: 69,
 151: 70,
 153: 71,
 157: 72,
 158: 73,
 161: 74,
 162: 75,
 163: 76,
 164: 77,
 165: 78,
 166: 79,
 167: 80,
 168: 81,
 169: 82,
 171: 83,
 172: 84,
 174: 85,
 175: 86,
 177: 87,
 178: 88,
 187: 89,
 189: 90,
 192: 91,
 197: 92,
 200: 93,
 202: 94,
 207: 95,
 208: 96,
 215: 97,
 218: 98,
 220: 99,
 221: 100,
 225: 101,
 230: 102,
 231: 103,
 232: 104,
 236: 

col_dict

In [38]:
lca_biodiesel_rapeseed.ia_spatial_dict

{1: 0,
 332: 1,
 333: 2,
 334: 3,
 335: 4,
 336: 5,
 337: 6,
 338: 7,
 339: 8,
 340: 9,
 341: 10,
 342: 11,
 343: 12,
 344: 13,
 345: 14,
 346: 15,
 347: 16,
 348: 17,
 349: 18,
 350: 19,
 351: 20,
 352: 21,
 353: 22,
 354: 23,
 355: 24,
 356: 25,
 357: 26,
 358: 27,
 359: 28,
 360: 29,
 361: 30,
 362: 31,
 363: 32,
 364: 33,
 365: 34,
 366: 35,
 367: 36,
 368: 37,
 369: 38,
 370: 39,
 371: 40,
 372: 41,
 373: 42,
 374: 43,
 375: 44,
 376: 45,
 377: 46,
 378: 47,
 379: 48,
 380: 49,
 381: 50,
 382: 51,
 383: 52,
 384: 53,
 385: 54,
 386: 55,
 387: 56,
 388: 57,
 389: 58,
 390: 59,
 391: 60,
 392: 61,
 393: 62,
 394: 63,
 395: 64,
 396: 65,
 397: 66,
 398: 67,
 399: 68,
 400: 69,
 401: 70,
 402: 71,
 403: 72,
 404: 73,
 405: 74,
 406: 75,
 407: 76,
 408: 77,
 409: 78,
 410: 79,
 411: 80,
 412: 81,
 413: 82,
 414: 83,
 415: 84,
 416: 85,
 417: 86,
 418: 87,
 419: 88,
 420: 89,
 421: 90,
 422: 91,
 423: 92,
 424: 93,
 425: 94,
 426: 95,
 427: 96,
 428: 97,
 429: 98,
 430: 99,
 431: 100,
 

### Outputs

In [39]:
geo_transform_params

array([(798, 513, 149, 181, 0, 8.0430989e+11, 8.0430989e+11, nan, nan, nan, nan, False),
       (800, 513, 150, 181, 0, 1.7383450e+12, 1.7383450e+12, nan, nan, nan, nan, False),
       (801, 513, 151, 181, 0, 9.8713508e+11, 9.8713508e+11, nan, nan, nan, nan, False),
       ...,
       (292, 346, 130,  15, 0, 1.2241805e+12, 1.2241805e+12, nan, nan, nan, nan, False),
       (238, 553, 107, 221, 0, 7.5642038e+11, 7.5642038e+11, nan, nan, nan, nan, False),
       (248, 416, 110,  85, 0, 3.9140363e+11, 3.9140363e+11, nan, nan, nan, nan, False)],
      dtype=[('geo_inv', '<u4'), ('geo_ia', '<u4'), ('row', '<u4'), ('col', '<u4'), ('uncertainty_type', 'u1'), ('amount', '<f4'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')])

In [40]:
print(geo_transform_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 68240 stored elements and shape (663, 228)>
  Coords	Values
  (1, 158)	347204124672.0
  (2, 171)	51837341696.0
  (3, 163)	8523996987392.0
  (4, 84)	131537625088.0
  (5, 72)	5155188736.0
  (6, 167)	448182452224.0
  (7, 60)	517429821440.0
  (8, 113)	123247525888.0
  (9, 103)	1134602878976.0
  (11, 101)	318635933696.0
  (12, 100)	64786063360.0
  (13, 28)	16896874250240.0
  (14, 127)	20662194176.0
  (15, 198)	468204912640.0
  (16, 64)	25475166208.0
  (18, 137)	17535100928.0
  (19, 217)	69315051520.0
  (20, 165)	301351272448.0
  (21, 197)	792793382912.0
  (22, 212)	1252236984320.0
  (23, 94)	571456356352.0
  (24, 82)	93154746368.0
  (25, 128)	330072227840.0
  (26, 91)	75011440640.0
  (27, 77)	467375587328.0
  :	:
  (662, 99)	13734757376.0
  (662, 100)	64786063360.0
  (662, 101)	318635933696.0
  (662, 105)	326369760.0
  (662, 107)	60333040.0
  (662, 110)	48416550912.0
  (662, 116)	64402984960.0
  (662, 124)	331677335552.0
  (662, 

# How to reverse the mapping of locations used for populating the different matrices ?

In [41]:
import geopandas as gp

In [42]:
lca_biodiesel_rapeseed.get_inventory_geocollections()

{'RoE', 'RoW', 'ecoinvent', 'world'}

In [43]:
reversed_geomapping = {v: k for k, v in bd.geomapping.items()}

In [44]:
reversed_geomapping

{1: 'GLO',
 2: 'CG',
 3: 'BR-SC',
 4: 'BA',
 5: 'IAI Area, EU27 & EFTA',
 6: 'BR-MS',
 7: 'CN-HB',
 8: 'CN-HN',
 9: 'CN-SX',
 10: 'CA-MB',
 11: 'BR',
 12: 'BR-PA',
 13: 'CN-JS',
 14: 'WEU',
 15: 'CA-QC',
 16: 'GR',
 17: 'TT',
 18: 'UZ',
 19: 'CN-SD',
 20: 'TH',
 21: 'UCTE',
 22: 'ER',
 23: 'ET',
 24: 'US-MN',
 25: 'BR-DF',
 26: 'TW',
 27: 'IN-TR',
 28: 'NO',
 29: 'LT',
 30: 'RU',
 31: 'SV',
 32: 'PG',
 33: 'RW',
 34: 'HK',
 35: 'CA-NU',
 36: 'KW',
 37: 'IE',
 38: 'BR-MA',
 39: 'IT',
 40: 'MZ',
 41: 'IN-HR',
 42: 'IN-WB',
 43: 'AO',
 44: 'UA',
 45: 'IAI Area, Russia & RER w/o EU27 & EFTA',
 46: 'US-OR',
 47: 'HU',
 48: 'CN-GX',
 49: 'AU-VIC',
 50: 'AU-NSW',
 51: 'AU-SA',
 52: 'CN-GD',
 53: 'Europe without Austria',
 54: 'MY',
 55: 'US-IL',
 56: 'PA',
 57: 'CM',
 58: 'CA-BC',
 59: 'CN-NM',
 60: 'SAS',
 61: 'US-MRO',
 62: 'CA-NT',
 63: 'IN-JK',
 64: 'MT',
 65: 'QA',
 66: 'MM',
 67: 'IR',
 68: 'IAI Area, South America',
 69: 'NE',
 70: 'HR',
 71: 'RAF',
 72: 'PH',
 73: 'KG',
 74: 'IN-MP',


## Examples

### Finding a specific location

In [45]:
[i for i in reversed_geomapping if ('FR') == reversed_geomapping[i]]

[144]

### Locations of different maps

World

In [47]:
reversed_geomapping[304]

'US'

Ecoinvent

In [48]:
reversed_geomapping[795]

('ecoinvent', 'AU-ACT')

RoW

In [50]:
reversed_geomapping[1142]

('RoW', 'RoW_147')

RoE

In [72]:
reversed_geomapping[1420]

('RoE', 'RoE_6')

Locations of the LCIA

In [73]:
reversed_geomapping[532]

('Freshwater-acidification-country', 'PN')

## Finding what are the locations associated to the lines and columns of the geotransform matrix

In [74]:
col_loc = {}
row_loc = {}

In [75]:
reversed_ia_mapping = {v: k for k, v in lca_biodiesel_rapeseed.ia_spatial_dict.items()}
reversed_inv_mapping = {v: k for k, v in lca_biodiesel_rapeseed.inv_spatial_dict.items()}

In [76]:
for i in range(0, len(lca_biodiesel_rapeseed.ia_spatial_dict)):
    col_num = list(lca_biodiesel_rapeseed.ia_spatial_dict.values())[i]
    loc = reversed_ia_mapping[col_num]
    mapped_loc = reversed_geomapping[loc]
    col_loc[col_num] = mapped_loc

for i in range(0, len(lca_biodiesel_rapeseed.inv_spatial_dict)):
    row_num = list(lca_biodiesel_rapeseed.inv_spatial_dict.values())[i]
    loc = reversed_inv_mapping[row_num]
    mapped_loc = reversed_geomapping[loc]
    row_loc[row_num] = mapped_loc

Locations of the LCIA

In [77]:
col_loc

{0: 'GLO',
 1: ('Freshwater-acidification-country', 'TO'),
 2: ('Freshwater-acidification-country', 'BE'),
 3: ('Freshwater-acidification-country', 'JM'),
 4: ('Freshwater-acidification-country', 'GQ'),
 5: ('Freshwater-acidification-country', 'BT'),
 6: ('Freshwater-acidification-country', 'IQ'),
 7: ('Freshwater-acidification-country', 'BM'),
 8: ('Freshwater-acidification-country', 'PF'),
 9: ('Freshwater-acidification-country', 'YE'),
 10: ('Freshwater-acidification-country', 'NP'),
 11: ('Freshwater-acidification-country', 'AF'),
 12: ('Freshwater-acidification-country', 'LS'),
 13: ('Freshwater-acidification-country', 'MA'),
 14: ('Freshwater-acidification-country', 'SC'),
 15: ('Freshwater-acidification-country', 'ZA'),
 16: ('Freshwater-acidification-country', 'KN'),
 17: ('Freshwater-acidification-country', 'GM'),
 18: ('Freshwater-acidification-country', 'EE'),
 19: ('Freshwater-acidification-country', 'LB'),
 20: ('Freshwater-acidification-country', 'KM'),
 21: ('Freshwater-

Locations of the LCI

In [78]:
row_loc

{0: 'GLO',
 1: 'CG',
 2: 'BA',
 3: 'BR',
 4: 'GR',
 5: 'TT',
 6: 'UZ',
 7: 'TH',
 8: 'ER',
 9: 'ET',
 10: 'TW',
 11: 'NO',
 12: 'LT',
 13: 'RU',
 14: 'SV',
 15: 'PG',
 16: 'RW',
 17: 'HK',
 18: 'KW',
 19: 'IE',
 20: 'IT',
 21: 'MZ',
 22: 'AO',
 23: 'UA',
 24: 'HU',
 25: 'MY',
 26: 'PA',
 27: 'CM',
 28: 'MT',
 29: 'QA',
 30: 'MM',
 31: 'IR',
 32: 'NE',
 33: 'HR',
 34: 'PH',
 35: 'KG',
 36: 'KH',
 37: 'KR',
 38: 'GI',
 39: 'LB',
 40: 'KZ',
 41: 'BW',
 42: 'BE',
 43: 'TG',
 44: 'ID',
 45: 'BJ',
 46: 'MN',
 47: 'CA',
 48: 'AZ',
 49: 'PL',
 50: 'SK',
 51: 'AM',
 52: 'IS',
 53: 'GB',
 54: 'AR',
 55: 'VE',
 56: 'KP',
 57: 'ES',
 58: 'TZ',
 59: 'EE',
 60: 'CY',
 61: 'CH',
 62: 'NI',
 63: 'OM',
 64: 'BG',
 65: 'XK',
 66: 'FR',
 67: 'CW',
 68: 'EC',
 69: 'SN',
 70: 'DO',
 71: 'CD',
 72: 'BH',
 73: 'GT',
 74: 'MK',
 75: 'SI',
 76: 'SG',
 77: 'PY',
 78: 'MU',
 79: 'IN',
 80: 'EG',
 81: 'CO',
 82: 'AL',
 83: 'SE',
 84: 'LU',
 85: 'AU',
 86: 'SS',
 87: 'HN',
 88: 'GA',
 89: 'DE',
 90: 'NL',
 91: 'TN

The element [0,0] of the geomapping matric corresponds to the intersection between the [GLO] location of the inventory and the [GLO] location of the LCIA

# How to reverse the mapping of activities used for populating the different matrices?

In [80]:
rev_act_dict, rev_product_dict, rev_bio_dict = lca_biodiesel_rapeseed.reverse_dict()

Mapping des activités selon les lignes de la inventory mapping matrix

In [82]:
rev_act_dict

{0: ('ecoinvent-3.10.1-cutoff', '1b4340043c90cb560250e2e8ea0819cb'),
 1: ('ecoinvent-3.10.1-cutoff', 'c293c136b707a2367c5eab264b2b11b8'),
 2: ('ecoinvent-3.10.1-cutoff', '5ebcfa78557d53672c9cd5fa93caf3ed'),
 3: ('ecoinvent-3.10.1-cutoff', '56564c43d9b890e5c84fc2bb2ac0c6e6'),
 4: ('ecoinvent-3.10.1-cutoff', '27c578cf4e0c31e9f43d493696cdaeaf'),
 5: ('ecoinvent-3.10.1-cutoff', 'bb2b7c43c0c84fae4ed5f68460dbaaec'),
 6: ('ecoinvent-3.10.1-cutoff', '9fe9f17b0c21043fe54ce8bbf578dc4b'),
 7: ('ecoinvent-3.10.1-cutoff', '8f77b0f094110eb4236f768ff6e86740'),
 8: ('ecoinvent-3.10.1-cutoff', 'b582589e5ef8b0b6ba541f67cc10a665'),
 9: ('ecoinvent-3.10.1-cutoff', 'f958f48ed9d3a9df295fb54d305c972b'),
 10: ('ecoinvent-3.10.1-cutoff', 'ea40387d4c17e79c8685971a379a3f0e'),
 11: ('ecoinvent-3.10.1-cutoff', '8db7d8a705866b02b68866f00a74ee12'),
 12: ('ecoinvent-3.10.1-cutoff', 'ae98cbce0ad02c65a3285a8c9bd0b61a'),
 13: ('ecoinvent-3.10.1-cutoff', 'e1f624ce4d8812ac730f3dc753f10ebf'),
 14: ('ecoinvent-3.10.1-cutoff

## Example

### Finding an activity in the activity database

Let's find the activities of the 23523, 23524, 23525 and 23526th lines of the inventory mapping mapping matrix

In [86]:
act_map = rev_act_dict[23523]
act_name = [act for act in bd.Database(act_map[0]) if act_map[1] == act['code']][0]
act_name

'Esterification - Rapeseed - MJ' (megajoule, FR, None)

In [87]:
act_map = rev_act_dict[23524]
act_name = [act for act in bd.Database(act_map[0]) if act_map[1] == act['code']][0]
act_name

'Esterification - Rapeseed' (kilogram, FR, None)

In [88]:
act_map = rev_act_dict[23525]
act_name = [act for act in bd.Database(act_map[0]) if act_map[1] == act['code']][0]
act_name

'Semi-refining - Rapeseed' (kilogram, FR, None)

In [90]:
act_map = rev_act_dict[23526]
act_name = [act for act in bd.Database(act_map[0]) if act_map[1] == act['code']][0]
act_name

'Oil extraction - Rapeseed' (kilogram, FR, None)

# How to reverse the mapping of the elementary flow in the characterization matrix

In [91]:
rev_act_dict, rev_product_dict, rev_bio_dict = lca_biodiesel_rapeseed.reverse_dict()

In [93]:
rev_bio_dict[1024]

('biosphere3', '9b00c244-fcce-4d38-922b-601a81f2ba34')

Reversing all the elementary flows corresponding to the columns of the reg cf matrix

In [94]:
rev_bio_dict

{0: ('biosphere3', '38a622c6-f086-4763-a952-7c6b3b1c42ba'),
 1: ('biosphere3', '541a823c-0aad-4dc4-9123-d4af4647d942'),
 2: ('biosphere3', '8cbaa905-41b0-4327-8403-bf1c8eb25429'),
 3: ('biosphere3', 'f681eb3c-854a-4f78-bcfe-76dfbcf9df3c'),
 4: ('biosphere3', 'a0fec60d-3f74-48bf-a2d2-58c30fc13e53'),
 5: ('biosphere3', 'd5649872-502b-4d23-b882-26d74a9fc4b4'),
 6: ('biosphere3', 'e2d860e3-1038-4386-a5f1-25ad75d18bbd'),
 7: ('biosphere3', '79a87f98-0b00-4e7e-86f8-70667307f696'),
 8: ('biosphere3', '90d374f1-b739-45b2-a734-d6bdd8c8dc9c'),
 9: ('biosphere3', 'cbf58f64-9286-406f-8dcb-ef1d81eec980'),
 10: ('biosphere3', 'c6310f81-3859-4c5c-990a-4662ba179693'),
 11: ('biosphere3', '9fbec2f4-6b42-4e63-9573-14e04befe20f'),
 12: ('biosphere3', '99c07b66-c039-4896-866d-06abdaa9d46a'),
 13: ('biosphere3', 'cdcdbfc9-3d5f-4e78-8033-8dbecbb2cfec'),
 14: ('biosphere3', '5ad58fcc-e9ba-4155-a3c9-e4ffb3065a6f'),
 15: ('biosphere3', 'eb23cf3a-b866-458d-94b3-9aee4145fcc9'),
 16: ('biosphere3', 'b9b68c0c-053e

In [95]:
char_act = {}

In [96]:
for i in range(0,len(rev_bio_dict)):
    flow = rev_bio_dict[i] # Equivalent to rev_bio_dict[bio_mapped_flox[i]]
    bio = [recherche for recherche in bd.Database('biosphere3') if recherche['code'] == flow[1]][0]
    char_act[i] = bio

## Examples

Finding the biosphere flows in the columns 663, 926, 1024, 1027, 1031, 1951, 2523, 2580 of the regionalized cf matrix (flows characterized by the method)

In [97]:
for i in [663, 926, 1024, 1027, 1031, 1951, 2523, 2580]:
    print(char_act[i])

'Ammonia' (kilogram, None, ('air',))
'Nitrate' (kilogram, None, ('air',))
'Sulfate' (kilogram, None, ('air',))
'Sulfur dioxide' (kilogram, None, ('air',))
'Sulfuric acid' (kilogram, None, ('air',))
'Nitric oxide' (kilogram, None, ('air',))
'Nitrogen oxides' (kilogram, None, ('air',))
'Sulfur trioxide' (kilogram, None, ('air',))
